In [5]:
import json
from PIL import Image
import os
import pandas as pd
import numpy as np

In [4]:
import supervision
import transformers
import pytorch_lightning

In [26]:
CSV_PATH = '../object_detection/data/train_solution_bounding_boxes.csv'
OUTPUT_JSON_PATH = '../object_detection/data/train_annotations_coco_bounding_boxes.json'
TRAIN_IMG_DIR = '../object_detection/data/training_images'

In [21]:
bounding_box = pd.read_csv(CSV_PATH)
display(bounding_box)

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


Preparing COCO FORMAT

In [23]:
# Prepare coco format
coco = {
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 1,
            "name": "car",
            "supercategory": "vehicle"
        }
    ]
}

In [25]:
#Map the image filename to image id
image_id_map = {}
annotation_id = 1
image_id_counter = 1

images = bounding_box['image'].unique()

In [27]:
for img_name in images:
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)
    if not os.path.exists(img_path):
        print(f'Warning: {img_path} not found!')
        continue
    with Image.open(img_path) as img:
        width, height = img.size
    # add image's info to coco
    image_info = {
        "file_name": img_name,
        "height": height,
        "width": width,
        "id": image_id_counter
    }
    coco['images'].append(image_info)
    image_id_map[img_name] = image_id_counter
    image_id_counter += 1

In [28]:
# process annotations
for idx, row in bounding_box.iterrows():
    img_name = row['image']
    if img_name not in image_id_map:
        continue # if the image is missing, then skip annotation
    x_min = row['xmin']
    y_min = row['ymin']
    x_max = row['xmax']
    y_max = row['ymax']

    # now to coco box format
    bbox_width = x_max - x_min
    bbox_height = y_max - y_min

    annotation = {
        "id": annotation_id,
        "image_id": image_id_map[img_name],
        "category_id": 1, # car
        "bbox": [x_min, y_min, bbox_width, bbox_height],
        "area": bbox_width * bbox_height,
        "iscrowd": 0,
        "segmentation": []
    }
    coco['annotations'].append(annotation)
    annotation_id += 1

a
